In [1]:
import pandas as pd
import numpy as np
import time
import turicreate as tc
from sklearn.cross_validation import train_test_split

/usr/local/lib/python2.7/dist-packages/sklearn/cross_validation.py:41: DeprecationWarning: This module was deprecated in version 0.18 in favor of the model_selection module into which all the refactored classes and functions are moved. Also note that the interface of the new CV iterators are different from that of this module. This module will be removed in 0.20.
  "This module will be removed in 0.20.", DeprecationWarning)


In [3]:
userItemData = pd.read_csv('data2.csv',encoding = 'unicode_escape',dtype='str')
#print(userItemData.head())
userList=list(set(userItemData["CustomerID"].tolist()))
#print(len(userList))
#print(userItemData.head(10))
filteredData=pd.DataFrame(columns=('customerId','products'))
rowCount=0
#print(userList) 
for user in userList: 
    products=[] 
    x=userItemData[userItemData.CustomerID==user]['StockCode'].tolist() 
    #print(x)
    s='|' 
    s=s.join(x) 
    filteredData.loc[rowCount]=[user,s] 
    rowCount+=1 
transactions=filteredData.dropna()
print(transactions.head())

  customerId                                           products
1      14176  21733|22645|22804|22810|21355|85123A|22968|211...
2      13305                     85061W|22855|22465|22740|21385
3      14679                                              84678
4      14911  22968|85071A|85071C|22355|21579|21576|22147|22...
5      14673  21485|22835|21143|22158|22633|22419|21212|2217...


In [4]:
customers = pd.DataFrame(transactions['customerId'])
print(type(transactions))

<class 'pandas.core.frame.DataFrame'>


In [5]:
transactions['products'] = transactions['products'].apply(lambda x: [str(i) for i in x.split('|')])
transactions.head(2).set_index('customerId')['products'].apply(pd.Series).reset_index()

/usr/local/lib/python2.7/dist-packages/ipykernel_launcher.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  """Entry point for launching an IPython kernel.


,customerId,0,1,2,3,4,5,6,7,8
0,14176,21733,22645,22804,22810,21355,85123A,22968,21109,21108
1,13305,85061W,22855,22465,22740,21385,NaN,NaN,NaN,NaN


In [6]:
pd.melt(transactions.head(2).set_index('customerId')['products'].apply(pd.Series).reset_index(), 
             id_vars=['customerId'],
             value_name='products') \
    .dropna().drop(['variable'], axis=1) \
    .groupby(['customerId', 'products']) \
    .agg({'products': 'count'}) \
    .rename(columns={'products': 'purchase_count'}) \
    .reset_index() \
    .rename(columns={'products': 'productId'})

,customerId,productId,purchase_count
0,13305,21385,1
1,13305,22465,1
2,13305,22740,1
3,13305,22855,1
4,13305,85061W,1
5,14176,21108,1
6,14176,21109,1
7,14176,21355,1
8,14176,21733,1
9,14176,22645,1


In [7]:
s=time.time()

data = pd.melt(transactions.set_index('customerId')['products'].apply(pd.Series).reset_index(), 
             id_vars=['customerId'],
             value_name='products') \
    .dropna().drop(['variable'], axis=1) \
    .groupby(['customerId', 'products']) \
    .agg({'products': 'count'}) \
    .rename(columns={'products': 'purchase_count'}) \
    .reset_index() \
    .rename(columns={'products': 'productId'})
data['productId'] = data['productId'].astype(np.str)

print("Execution time:", round((time.time()-s)/60,2), "minutes")

('Execution time:', 0.01, 'minutes')


In [8]:
print(data.shape)
data.head()

(12094, 3)


,customerId,productId,purchase_count
0,12347,20780,1
1,12347,20782,1
2,12347,21064,1
3,12347,21171,1
4,12347,21731,1


In [9]:
def create_data_dummy(data):
    data_dummy = data.copy()
    data_dummy['purchase_dummy'] = 1
    return data_dummy

In [10]:
data_dummy = create_data_dummy(data)

In [11]:
df_matrix = pd.pivot_table(data, values='purchase_count', index='customerId', columns='productId')
df_matrix.head()

productId,10002,10120,10123C,10124A,10124G,10125,10133,10135,11001,15034,...,90214J,90214K,90214M,90214S,90214V,BANK CHARGES,C2,D,M,POST
customerId,,,,,,,,,,,,,,,,,,,,,
12347,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
12386,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
12395,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,1.0
12427,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,1.0
12431,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [12]:
df_matrix_norm = (df_matrix-df_matrix.min())/(df_matrix.max()-df_matrix.min())
print(df_matrix_norm.shape)
df_matrix_norm.head()

(559, 1969)


productId,10002,10120,10123C,10124A,10124G,10125,10133,10135,11001,15034,...,90214J,90214K,90214M,90214S,90214V,BANK CHARGES,C2,D,M,POST
customerId,,,,,,,,,,,,,,,,,,,,,
12347,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
12386,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
12395,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0.0
12427,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0.0
12431,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [13]:
d = df_matrix_norm.reset_index()
d.index.names = ['scaled_purchase_freq']
data_norm = pd.melt(d, id_vars=['customerId'], value_name='scaled_purchase_freq').dropna()
print(data_norm.shape)
data_norm.head()

(6476, 3)


,customerId,productId,scaled_purchase_freq
3386,12748,10133,1.0
3680,15983,10133,0.0
3801,17259,10133,0.0
3901,18118,10133,0.0
7847,12682,15056BL,0.0


In [14]:
def normalize_data(data):
    df_matrix = pd.pivot_table(data, values='purchase_count', index='customerId', columns='productId')
    df_matrix_norm = (df_matrix-df_matrix.min())/(df_matrix.max()-df_matrix.min())
    d = df_matrix_norm.reset_index()
    d.index.names = ['scaled_purchase_freq']
    return pd.melt(d, id_vars=['customerId'], value_name='scaled_purchase_freq').dropna()

In [15]:
train, test = train_test_split(data, test_size = .2)
print(train.shape, test.shape)

((9675, 3), (2419, 3))


In [16]:

train_data = tc.SFrame(train)
test_data = tc.SFrame(test)

In [17]:
train_data

customerId,productId,purchase_count
16539,22855,1
15601,21520,1
17341,21376,1
16150,22969,1
13715,21042,1
17596,22551,1
14213,22942,1
13523,22180,1
12967,21086,1
16016,21916,1


In [18]:
test_data

customerId,productId,purchase_count
15716,22572,1
14307,22914,1
12921,22748,1
13013,21216,1
14449,22383,1
16140,22409,1
17884,22570,1
16891,22851,1
14404,22423,1
16402,22638,1


In [19]:
def split_data(data):
    '''
    Splits dataset into training and test set.
    
    Args:
        data (pandas.DataFrame)
        
    Returns
        train_data (tc.SFrame)
        test_data (tc.SFrame)
    '''
    train, test = train_test_split(data, test_size = .2)
    train_data = tc.SFrame(train)
    test_data = tc.SFrame(test)
    return train_data, test_data

In [20]:
train_data_dummy, test_data_dummy = split_data(data_dummy)
train_data_norm, test_data_norm = split_data(data_norm)

In [21]:
user_id = 'customerId'
item_id = 'productId'
target = 'purchase_count'
users_to_recommend = list(transactions[user_id])
n_rec = 10 # number of items to recommend
n_display = 30

In [22]:
popularity_model = tc.popularity_recommender.create(train_data, 
                                                    user_id=user_id, 
                                                    item_id=item_id, 
                                                    target=target)

Preparing data set.

Data has 9675 observations with 553 users and 1841 items.

Data prepared in: 0.108856s

9675 observations to process; with 1841 unique items.

In [23]:
# Get recommendations for a list of users to recommend (from customers file)
# Printed below is head / top 30 rows for first 3 customers with 10 recommendations each

popularity_recomm = popularity_model.recommend(users=users_to_recommend, k=n_rec)
popularity_recomm.print_rows(n_display)

+------------+-----------+---------------+------+
| customerId | productId |     score     | rank |
+------------+-----------+---------------+------+
|   14176    |   22803   |      8.0      |  1   |
|   14176    |   21068   | 5.66666666667 |  2   |
|   14176    |   21448   |      5.0      |  3   |
|   14176    |   84827   |      4.0      |  4   |
|   14176    |   21071   | 3.33333333333 |  5   |
|   14176    |   21730   | 3.28571428571 |  6   |
|   14176    |   21650   |      3.0      |  7   |
|   14176    |   21651   |      3.0      |  8   |
|   14176    |   21415   |      3.0      |  9   |
|   14176    |   90082B  |      3.0      |  10  |
|   13305    |   22803   |      8.0      |  1   |
|   13305    |   21068   | 5.66666666667 |  2   |
|   13305    |   21448   |      5.0      |  3   |
|   13305    |   84827   |      4.0      |  4   |
|   13305    |   21071   | 3.33333333333 |  5   |
|   13305    |   21730   | 3.28571428571 |  6   |
|   13305    |   21650   |      3.0      |  7   |


In [24]:
def model(train_data, name, user_id, item_id, target, users_to_recommend, n_rec, n_display):
    if name == 'popularity':
        model = tc.popularity_recommender.create(train_data, 
                                                    user_id=user_id, 
                                                    item_id=item_id, 
                                                    target=target)
    elif name == 'cosine':
        model = tc.item_similarity_recommender.create(train_data, 
                                                    user_id=user_id, 
                                                    item_id=item_id, 
                                                    target=target, 
                                                    similarity_type='cosine')
    elif name == 'pearson':
        model = tc.item_similarity_recommender.create(train_data, 
                                                    user_id=user_id, 
                                                    item_id=item_id, 
                                                    target=target, 
                                                    similarity_type='pearson')
        
    recom = model.recommend(users=users_to_recommend, k=n_rec)
    recom.print_rows(n_display)
    return model

In [25]:
user_id = 'customerId'
item_id = 'productId'
users_to_recommend = list(customers[user_id])
n_rec = 10 # number of items to recommend
n_display = 30 # to print the head / first few rows in a defined dataset

In [26]:
name = 'popularity'
target = 'purchase_dummy'
pop_dummy = model(train_data_dummy, name, user_id, item_id, target, users_to_recommend, n_rec, n_display)

Warning: Ignoring columns purchase_count;

To use these columns in scoring predictions, use a model that allows the use of additional features.

Preparing data set.

Data has 9675 observations with 553 users and 1857 items.

Data prepared in: 0.066335s

9675 observations to process; with 1857 unique items.

+------------+-----------+-------+------+
| customerId | productId | score | rank |
+------------+-----------+-------+------+
|   14176    |   22837   |  1.0  |  1   |
|   14176    |   22635   |  1.0  |  2   |
|   14176    |   20679   |  1.0  |  3   |
|   14176    |   82582   |  1.0  |  4   |
|   14176    |   21467   |  1.0  |  5   |
|   14176    |   21930   |  1.0  |  6   |
|   14176    |   84347   |  1.0  |  7   |
|   14176    |   21464   |  1.0  |  8   |
|   14176    |   22800   |  1.0  |  9   |
|   14176    |   22144   |  1.0  |  10  |
|   13305    |   22837   |  1.0  |  1   |
|   13305    |   22635   |  1.0  |  2   |
|   13305    |   20679   |  1.0  |  3   |
|   13305    |   82582   |  1.0  |  4   |
|   13305    |   21467   |  1.0  |  5   |
|   13305    |   21930   |  1.0  |  6   |
|   13305    |   84347   |  1.0  |  7   |
|   13305    |   21464   |  1.0  |  8   |
|   13305    |   22800   |  1.0  |  9   |
|   13305    |   22144   |  1.0  |  10  |
|   14679    |   22837   |  1.0  |

In [27]:
name = 'popularity'
target = 'scaled_purchase_freq'
pop_norm = model(train_data_norm, name, user_id, item_id, target, users_to_recommend, n_rec, n_display)

Preparing data set.

Data has 5180 observations with 519 users and 502 items.

Data prepared in: 0.013839s

5180 observations to process; with 502 unique items.

+------------+-----------+----------------+------+
| customerId | productId |     score      | rank |
+------------+-----------+----------------+------+
|   14176    |   47471   |      1.0       |  1   |
|   14176    |   22649   |      1.0       |  2   |
|   14176    |   90082D  |      1.0       |  3   |
|   14176    |   22211   |      1.0       |  4   |
|   14176    |   22200   |      1.0       |  5   |
|   14176    |   84625C  |      1.0       |  6   |
|   14176    |   22535   |      1.0       |  7   |
|   14176    |   21192   | 0.666666666667 |  8   |
|   14176    |   20986   | 0.666666666667 |  9   |
|   14176    |   35953   | 0.666666666667 |  10  |
|   13305    |   47471   |      1.0       |  1   |
|   13305    |   22649   |      1.0       |  2   |
|   13305    |   90082D  |      1.0       |  3   |
|   13305    |   22211   |      1.0       |  4   |
|   13305    |   22200   |      1.0       |  5   |
|   13305    |   84625C  |      1.0       |  6   |
|   13305    |   22535   |     

In [28]:
train.groupby(by=item_id)['purchase_count'].mean().sort_values(ascending=False).head(20)

productId
22803     8.000000
21068     5.666667
21448     5.000000
84827     4.000000
21071     3.333333
21730     3.285714
21785     3.000000
90082B    3.000000
21651     3.000000
21650     3.000000
90199C    3.000000
21415     3.000000
21871     2.857143
79000     2.500000
37370     2.363636
70007     2.250000
82483     2.166667
21090     2.000000
21812     2.000000
47580     2.000000
Name: purchase_count, dtype: float64

In [29]:
name = 'cosine'
target = 'purchase_count'
cos = model(train_data, name, user_id, item_id, target, users_to_recommend, n_rec, n_display)

Preparing data set.

Data has 9675 observations with 553 users and 1841 items.

Data prepared in: 0.025153s

Training model from provided data.

Gathering per-item and per-user statistics.

...

+-------------------------------------+------------------+-----------------+

Finalizing lookup tables.

Generating candidate set for working with new users.

Finished training in 1.12507s

+------------+-----------+-----------------+------+
| customerId | productId |      score      | rank |
+------------+-----------+-----------------+------+
|   14176    |   22809   |  0.107370361686 |  1   |
|   14176    |   21110   |  0.106541872025 |  2   |
|   14176    |   22073   |  0.100222967565 |  3   |
|   14176    |   22068   | 0.0984775349498 |  4   |
|   14176    |   22513   | 0.0944911241531 |  5   |
|   14176    |   71496A  | 0.0944911241531 |  6   |
|   14176    |   78124   | 0.0944911241531 |  7   |
|   14176    |   84509A  | 0.0913298279047 |  8   |
|   14176    |   21358   | 0.0818317085505 |  9   |
|   14176    |   82581   | 0.0804065689445 |  10  |
|   13305    |   72351B  |  0.115470051765 |  1   |
|   13305    |   22680   |  0.115470051765 |  2   |
|   13305    |   22252   |       0.1       |  3   |
|   13305    |   21640   |       0.1       |  4   |
|   13305    |   22610   |       0.1       |  5   |
|   13305    |   21638   |       0.1       |  6   |
|   13305   

In [30]:
name = 'cosine'
target = 'purchase_dummy'
cos_dummy = model(train_data_dummy, name, user_id, item_id, target, users_to_recommend, n_rec, n_display)

Warning: Ignoring columns purchase_count;

To use these columns in scoring predictions, use a model that allows the use of additional features.

Preparing data set.

Data has 9675 observations with 553 users and 1857 items.

Data prepared in: 0.045359s

...

+-------------------------------------+------------------+-----------------+

Finalizing lookup tables.

Generating candidate set for working with new users.

Finished training in 0.060818s

+------------+-----------+-----------------+------+
| customerId | productId |      score      | rank |
+------------+-----------+-----------------+------+
|   14176    |   21896   |  0.111506700516 |  1   |
|   14176    |   21377   |  0.111506700516 |  2   |
|   14176    |   21205   |  0.111506700516 |  3   |
|   14176    |   21375   |  0.111506700516 |  4   |
|   14176    |   20618   |  0.111506700516 |  5   |
|   14176    |   21558   |  0.106765576771 |  6   |
|   14176    |   21356   |  0.101263982909 |  7   |
|   14176    |   21865   | 0.0969926629748 |  8   |
|   14176    |   22808   | 0.0824786084039 |  9   |
|   14176    |   85187   | 0.0824786084039 |  10  |
|   13305    |   22116   |  0.166666666667 |  1   |
|   13305    |   85129C  |  0.166666666667 |  2   |
|   13305    |   85130A  |  0.166666666667 |  3   |
|   13305    |   46000P  |  0.13608276844  |  4   |
|   13305    |   10124G  |  0.13608276844  |  5   |
|   13305    |   21638   |  0.13608276844  |  6   |
|   13305   

In [31]:
name = 'cosine'
target = 'scaled_purchase_freq'
cos_norm = model(train_data_norm, name, user_id, item_id, target, users_to_recommend, n_rec, n_display)

Preparing data set.

Data has 5180 observations with 519 users and 502 items.

Data prepared in: 0.013077s

Training model from provided data.

Gathering per-item and per-user statistics.

...

+------------+-----------+------------------+------+
| customerId | productId |      score       | rank |
+------------+-----------+------------------+------+
|   14176    |   85231B  |       0.0        |  1   |
|   14176    |   21889   |       0.0        |  2   |
|   14176    |   22865   |       0.0        |  3   |
|   14176    |   22568   |       0.0        |  4   |
|   14176    |   22727   |       0.0        |  5   |
|   14176    |   20975   |       0.0        |  6   |
|   14176    |   22082   |       0.0        |  7   |
|   14176    |   85123A  |       0.0        |  8   |
|   14176    |   22927   |       0.0        |  9   |
|   14176    |   84949   |       0.0        |  10  |
|   13305    |   72800E  | 0.00772116541862 |  1   |
|   13305    |   21982   | 0.00742901444435 |  2   |
|   13305    |   22554   | 0.00725902199745 |  3   |
|   13305    |   72799E  | 0.00660726070404 |  4   |
|   13305    |   72131   | 0.00660726070404 |  5   |
|   13305    |   20757   | 0.00660726070404 | 

+-------------------------------------+------------------+-----------------+

Finalizing lookup tables.

Generating candidate set for working with new users.

Finished training in 0.010272s

In [32]:
name = 'pearson'
target = 'purchase_count'
pear = model(train_data, name, user_id, item_id, target, users_to_recommend, n_rec, n_display)

Preparing data set.

Data has 9675 observations with 553 users and 1841 items.

Data prepared in: 0.080119s

Training model from provided data.

Gathering per-item and per-user statistics.

...

+-------------------------------------+------------------+-----------------+

Finalizing lookup tables.

Generating candidate set for working with new users.

Finished training in 0.039872s

+------------+-----------+---------------+------+
| customerId | productId |     score     | rank |
+------------+-----------+---------------+------+
|   14176    |   22803   |      8.0      |  1   |
|   14176    |   21068   | 5.66666666667 |  2   |
|   14176    |   21448   |      5.0      |  3   |
|   14176    |   84827   |      4.0      |  4   |
|   14176    |   21071   | 3.33324073752 |  5   |
|   14176    |   21730   | 3.28571428571 |  6   |
|   14176    |   21650   |      3.0      |  7   |
|   14176    |   21651   |      3.0      |  8   |
|   14176    |   21415   |      3.0      |  9   |
|   14176    |   90082B  |      3.0      |  10  |
|   13305    |   22803   |      8.0      |  1   |
|   13305    |   21068   | 5.59316350381 |  2   |
|   13305    |   21448   |      5.0      |  3   |
|   13305    |   84827   |      4.0      |  4   |
|   13305    |   21071   | 3.24164468726 |  5   |
|   13305    |   21730   | 3.19138895665 |  6   |
|   13305    |   21650   |      3.0      |  7   |


In [33]:
name = 'pearson'
target = 'purchase_dummy'
pear_dummy = model(train_data_dummy, name, user_id, item_id, target, users_to_recommend, n_rec, n_display)

Warning: Ignoring columns purchase_count;

To use these columns in scoring predictions, use a model that allows the use of additional features.

Preparing data set.

Data has 9675 observations with 553 users and 1857 items.

Data prepared in: 0.022555s

...

+-------------------------------------+------------------+-----------------+

Finalizing lookup tables.

+------------+-----------+-------+------+
| customerId | productId | score | rank |
+------------+-----------+-------+------+
|   14176    |   22837   |  0.0  |  1   |
|   14176    |   22635   |  0.0  |  2   |
|   14176    |   20679   |  0.0  |  3   |
|   14176    |   82582   |  0.0  |  4   |
|   14176    |   21467   |  0.0  |  5   |
|   14176    |   21930   |  0.0  |  6   |
|   14176    |   84347   |  0.0  |  7   |
|   14176    |   21464   |  0.0  |  8   |
|   14176    |   22800   |  0.0  |  9   |
|   14176    |   22144   |  0.0  |  10  |
|   13305    |   22837   |  0.0  |  1   |
|   13305    |   22635   |  0.0  |  2   |
|   13305    |   20679   |  0.0  |  3   |
|   13305    |   82582   |  0.0  |  4   |
|   13305    |   21467   |  0.0  |  5   |
|   13305    |   21930   |  0.0  |  6   |
|   13305    |   84347   |  0.0  |  7   |
|   13305    |   21464   |  0.0  |  8   |
|   13305    |   22800   |  0.0  |  9   |
|   13305    |   22144   |  0.0  |  10  |
|   14679    |   22837   |  0.0  |

Generating candidate set for working with new users.

Finished training in 0.043527s

In [34]:
name = 'pearson'
target = 'scaled_purchase_freq'
pear_norm = model(train_data_norm, name, user_id, item_id, target, users_to_recommend, n_rec, n_display)

Preparing data set.

Data has 5180 observations with 519 users and 502 items.

Data prepared in: 0.08879s

Training model from provided data.

Gathering per-item and per-user statistics.

...

+-------------------------------------+------------------+-----------------+

Finalizing lookup tables.

Generating candidate set for working with new users.

Finished training in 0.02778s

+------------+-----------+----------------+------+
| customerId | productId |     score      | rank |
+------------+-----------+----------------+------+
|   14176    |   47471   |      1.0       |  1   |
|   14176    |   22649   |      1.0       |  2   |
|   14176    |   90082D  |      1.0       |  3   |
|   14176    |   22211   |      1.0       |  4   |
|   14176    |   22200   |      1.0       |  5   |
|   14176    |   84625C  |      1.0       |  6   |
|   14176    |   22535   |      1.0       |  7   |
|   14176    |   21192   | 0.666666666667 |  8   |
|   14176    |   20986   | 0.666666666667 |  9   |
|   14176    |   35953   | 0.666666666667 |  10  |
|   13305    |   47471   |      1.0       |  1   |
|   13305    |   22649   |      1.0       |  2   |
|   13305    |   90082D  |      1.0       |  3   |
|   13305    |   22211   |      1.0       |  4   |
|   13305    |   22200   |      1.0       |  5   |
|   13305    |   84625C  |      1.0       |  6   |
|   13305    |   22535   |     

In [35]:
models_w_counts = [popularity_model, cos, pear]
models_w_dummy = [pop_dummy, cos_dummy, pear_dummy]
models_w_norm = [pop_norm, cos_norm, pear_norm]

names_w_counts = ['Popularity Model on Purchase Counts', 'Cosine Similarity on Purchase Counts', 'Pearson Similarity on Purchase Counts']
names_w_dummy = ['Popularity Model on Purchase Dummy', 'Cosine Similarity on Purchase Dummy', 'Pearson Similarity on Purchase Dummy']
names_w_norm = ['Popularity Model on Scaled Purchase Counts', 'Cosine Similarity on Scaled Purchase Counts', 'Pearson Similarity on Scaled Purchase Counts']

In [36]:
eval_counts = tc.recommender.util.compare_models(test_data, models_w_counts, model_names=names_w_counts)

PROGRESS: Evaluate model Popularity Model on Purchase Counts

Precision and recall summary statistics by cutoff
+--------+-------------------+-------------------+
| cutoff |   mean_precision  |    mean_recall    |
+--------+-------------------+-------------------+
|   1    |  0.00213675213675 | 0.000125691302162 |
|   2    |  0.00106837606838 | 0.000125691302162 |
|   3    | 0.000712250712251 | 0.000125691302162 |
|   4    |  0.00106837606838 | 0.000363108206245 |
|   5    | 0.000854700854701 | 0.000363108206245 |
|   6    |  0.00178062678063 |  0.0036394614826  |
|   7    |  0.0018315018315  |  0.00406681190995 |
|   8    |  0.00160256410256 |  0.00406681190995 |
|   9    |  0.0014245014245  |  0.00406681190995 |
|   10   |  0.00149572649573 |  0.00428048712362 |
+--------+-------------------+-------------------+
[10 rows x 3 columns]


Overall RMSE: 0.739923182125

Per User RMSE (best)
+------------+-------+------+
| customerId | count | rmse |
+------------+-------+------+
|   17857

In [37]:
eval_dummy = tc.recommender.util.compare_models(test_data_dummy, models_w_dummy, model_names=names_w_dummy)

PROGRESS: Evaluate model Popularity Model on Purchase Dummy

Precision and recall summary statistics by cutoff
+--------+------------------+------------------+
| cutoff |  mean_precision  |   mean_recall    |
+--------+------------------+------------------+
|   1    |       0.0        |       0.0        |
|   2    | 0.00108459869848 | 0.00108459869848 |
|   3    | 0.00289226319595 | 0.00158423463738 |
|   4    | 0.0027114967462  | 0.00172884779718 |
|   5    | 0.00216919739696 | 0.00172884779718 |
|   6    | 0.00216919739696 | 0.00183730766703 |
|   7    | 0.00216919739696 | 0.00214719300945 |
|   8    | 0.00216919739696 | 0.00234439277281 |
|   9    | 0.00216919739696 | 0.0027782322522  |
|   10   | 0.00195227765727 | 0.0027782322522  |
+--------+------------------+------------------+
[10 rows x 3 columns]


Overall RMSE: 0.0

Per User RMSE (best)
+------------+-------+------+
| customerId | count | rmse |
+------------+-------+------+
|   15983    |   9   | 0.0  |
+------------+-----

In [38]:
eval_norm = tc.recommender.util.compare_models(test_data_norm, models_w_norm, model_names=names_w_norm)

PROGRESS: Evaluate model Popularity Model on Scaled Purchase Counts

Precision and recall summary statistics by cutoff
+--------+------------------+-------------------+
| cutoff |  mean_precision  |    mean_recall    |
+--------+------------------+-------------------+
|   1    | 0.00251256281407 | 0.000251256281407 |
|   2    | 0.00376884422111 |  0.00234505862647 |
|   3    | 0.00335008375209 |  0.00276381909548 |
|   4    | 0.00314070351759 |  0.00288346494377 |
|   5    | 0.00402010050251 |  0.00439100263221 |
|   6    | 0.00376884422111 |  0.00522852357023 |
|   7    | 0.00430725053841 |  0.0060200268743  |
|   8    | 0.00408291457286 |  0.00664816757782 |
|   9    | 0.00390843104411 |  0.00692734122382 |
|   10   | 0.0035175879397  |  0.00692734122382 |
+--------+------------------+-------------------+
[10 rows x 3 columns]


Overall RMSE: 0.352527142163

Per User RMSE (best)
+------------+-------+------+
| customerId | count | rmse |
+------------+-------+------+
|   13786    |  

In [39]:
users_to_recommend = list(customers[user_id])

final_model = tc.item_similarity_recommender.create(tc.SFrame(data_dummy), 
                                            user_id=user_id, 
                                            item_id=item_id, 
                                            target='purchase_dummy', 
                                            similarity_type='cosine')

recom = final_model.recommend(users=users_to_recommend, k=n_rec)
recom.print_rows(n_display)

Warning: Ignoring columns purchase_count;

To use these columns in scoring predictions, use a model that allows the use of additional features.

Preparing data set.

Data has 12094 observations with 559 users and 1969 items.

Data prepared in: 0.023404s

...

+-------------------------------------+------------------+-----------------+

Finalizing lookup tables.

Generating candidate set for working with new users.

Finished training in 0.083465s

+------------+-----------+-----------------+------+
| customerId | productId |      score      | rank |
+------------+-----------+-----------------+------+
|   14176    |   84709B  | 0.0828823844592 |  1   |
|   14176    |   22068   | 0.0828823844592 |  2   |
|   14176    |   78124   | 0.0812797678841 |  3   |
|   14176    |   71496A  | 0.0812797678841 |  4   |
|   14176    |   71495B  | 0.0812797678841 |  5   |
|   14176    |   22513   | 0.0812797678841 |  6   |
|   14176    |   21201   | 0.0792641573482 |  7   |
|   14176    |   84509A  | 0.0746109551854 |  8   |
|   14176    |   21205   | 0.0728127360344 |  9   |
|   14176    |   20618   | 0.0728127360344 |  10  |
|   13305    |   22116   |  0.170710682869 |  1   |
|   13305    |   85129C  |  0.170710682869 |  2   |
|   13305    |   85130A  |  0.139384686947 |  3   |
|   13305    |   21333   |  0.136251199245 |  4   |
|   13305    |   21179   |  0.120710682869 |  5   |
|   13305    |   21640   |       0.1       |  6   |
|   13305   

In [40]:
df_rec = recom.to_dataframe()
print(df_rec.shape)
df_rec.head()

(5590, 4)


,customerId,productId,score,rank
0,14176,84709B,0.082882,1
1,14176,22068,0.082882,2
2,14176,78124,0.081280,3
3,14176,71496A,0.081280,4
4,14176,71495B,0.081280,5


In [41]:
df_rec['recommendedProducts'] = df_rec.groupby([user_id])[item_id].transform(lambda x: '|'.join(x.astype(str)))
df_output = df_rec[['customerId', 'recommendedProducts']].drop_duplicates().sort_values('customerId').set_index('customerId')
df_output.head()

,recommendedProducts
customerId,
12347,22730|84952A|21846|84997A|20655|21672|85068|47...
12386,22553|22515|21534|22551|84658|84247E|84534B|20...
12395,84991|22951|85170D|22134|22949|21213|22932|211...
12427,21327|21785|21650|21209|21167|21090|20839|2077...
12431,22728|22729|22725|22730|20655|15056N|47563A|21...


In [43]:
df_output.to_csv('output/option1_recommendation.csv')
print("An output file can be found in 'output' folder with name 'option1_recommendation.csv'")

An output file can be found in 'output' folder with name 'option1_recommendation.csv'


In [54]:
def customer_recomendation(customer_id):
    if customer_id not in df_output.index:
        print('Customer not found.')
        return customer_id
   # return df_output.loc[customer_id]
    else:
        desc=df_output.loc[customer_id]["recommendedProducts"]
        prod=list(desc.split("|"))
        recomproducts=[]
        for each in prod:
            z=list(set(userItemData[userItemData.StockCode==each]["Description"].tolist()))
            recomproducts.append(z[0])
        return recomproducts

In [55]:
customer_recomendation('12431')

[u'ALARM CLOCK BAKELIKE PINK',
 u'ALARM CLOCK BAKELIKE ORANGE',
 u'ALARM CLOCK BAKELIKE CHOCOLATE',
 u'ALARM CLOCK BAKELIKE IVORY',
 u'QUEEN OF SKIES LUGGAGE TAG',
 u'EDWARDIAN PARASOL NATURAL',
 u'RETRO LONGBOARD IRONING BOARD COVER',
 u"POTTING SHED SOW 'N' GROW SET",
 u'MIRRORED WALL ART FOXY',
 u'BIRTHDAY CARD, RETRO SPOT']